# Лабораторная работа №1

_Выполнили: Гуревич Михаил и Трохан Александр, M33001_

In [61]:
from typing import Literal

import json

import numpy as np
from prettytable import PrettyTable

Вспомогательные функции:

In [62]:
TYPES = {
    "leq": "≤",
    "geq": "≥",
    "eq": "="
}


def append_to_problem(string: str, coef: float, i: int):
    if coef != 0:
        if len(string) > 0:
            if coef == 1:
                string += f" + x_{i + 1}"
            elif coef == -1:
                string += f" - x_{i + 1}"
            elif coef > 0:
                string += f" + {coef}x_{i + 1}"
            else:
                string += f" - {-coef}x_{i + 1}"
        
        else:
            if coef == 1:
                string += f"x_{i + 1}"
            elif coef == -1:
                string += f"-x_{i + 1}"
            else:
                string += f"{coef}x_{i + 1}"
        
    return string
        
def type_to_string(type: str):
    return TYPES[type]

In [63]:
class Constraint:
    """Constraint class for linear programming problem"""

    def __init__(self, coefs: list[float], type: Literal["eq", "leq", "geq"], b: float):
        if type not in {"leq", "geq", "eq"}:
            raise ValueError("Type must be one of \"leq\", \"geq\", \"eq\"")
        if len(coefs) == 0:
            raise ValueError("Coefs must not be empty")

        self.coefs = coefs
        self.type = type
        self.b = b     

    def __call__(self, x: list[float]):
        lhs = np.dot(self.coefs, x)
        if self.type == "leq":
            return lhs <= self.b
        elif self.type == "geq":
            return lhs >= self.b
        else:
            return lhs == self.b
        
    def __str__(self):
        string = ""
        for i, coef in enumerate(self.coefs):
            string = append_to_problem(string, coef, i)
        string += f" {type_to_string(self.type)} {self.b}"
        return string


class LinearProgram:
    """Linear programming problem class"""

    @staticmethod
    def validate(f: list[float], goal: Literal["min", "max"], constraints: list[Constraint]):
        if len(f) == 0 or not all(f):
            raise ValueError("Function must not be empty")
        if goal not in {"min", "max"}:
            raise ValueError("Goal must be one of \"min\", \"max\"")
        if len(constraints) == 0:
            raise ValueError("Constraints must not be empty")
        for constraint in constraints:
            if len(constraint.coefs) != len(constraints[0].coefs) or len(constraint.coefs) != len(f) or not all(constraint.coefs):
                raise ValueError("All constraints must have the same number of coefficients equal to the number of variables in the function")

    def __init__(self, f: list[float], goal: Literal["min", "max"], constraints: list[Constraint]):
        LinearProgram.validate(f, goal, constraints)

        self.f = f
        self.goal = goal
        self.constraints = constraints

    def __str__(self):
        result = "Problem:\n"
        
        problem = ""
        for i, coef in enumerate(self.f):
            problem = append_to_problem(problem, coef, i)
        problem += f" -> {self.goal}\n"

        result += f"{problem}\nConstraints:\n"

        for constraint in self.constraints:
            string = ""
            for i, coef in enumerate(constraint.coefs):
                string = append_to_problem(string, coef, i)
            string += f" {type_to_string(constraint.type)} {constraint.b}"
            result += f"{string}\n"

        return result

In [64]:
with open("task.json") as f:
    data = json.loads(f.read())
    constraints = list(map(lambda c: Constraint(**c), data["constraints"]))
    problem = LinearProgram(data["f"], data["goal"], constraints)
print(problem)

Problem:
x_1 + x_2 -> max

Constraints:
2x_1 + x_2 ≤ 4
x_1 + 2x_2 ≥ 3



In [75]:
class SimplexTable:
    """Simplex table class"""

    def __init__(self):
        self.table = []
        self.basic_variables = []

    def add_row(self, basis: int, row: list[float]):
        self.basic_variables.append(basis)
        self.table.append(row)

    def subtract_rows(self, row1: int, row2: int):
        """Subtracts row2 from row1"""
        self.table[row1] = np.subtract(self.table[row1], self.table[row2]).tolist()

    def __getitem__(self, key: int):
        """Returns row with given index"""
        return self.table[key: int]

    def print(self):
        table = PrettyTable()
        table.field_names = ["Basis"] + [f"x_{i + 1}" for i in range(len(self.table[0]) - 1)] + ["b"]
        for i, row in enumerate(self.table):
            if (self.basic_variables[i] == -1):
                table.add_row(["f"] + row)
            else:
                table.add_row([f"x_{self.basic_variables[i] + 1}"] + row)
        
        print(table)


class SimplexSolver:
    """2-phase simplex method solver"""

    def __init__(self, problem: LinearProgram):
        self.problem = problem

    def solve(self, disp: bool=False):
        self.disp = disp

        if self.problem.goal == "min":
            self.problem.f = np.multiply(self.problem.f, -1)
            self.problem.goal = "max"

        self._phase1()

    def _phase1(self):
        # eliminate negative b
        for constraint in self.problem.constraints:
            if constraint.b < 0:
                constraint.coefs = np.multiply(constraint.coefs, -1).tolist()
                constraint.b *= -1
                if constraint.type == "leq":
                    constraint.type = "geq"
                elif constraint.type == "geq":
                    constraint.type = "leq"

        geqs = len(list(filter(lambda c: c.type == "geq", self.problem.constraints)))
        leqs = len(list(filter(lambda c: c.type == "leq", self.problem.constraints)))
        eqs = len(list(filter(lambda c: c.type == "eq", self.problem.constraints)))

        self.simplex_table = SimplexTable()
        cur_geqs, cur_leqs, cur_eqs, initial_count = 0, 0, 0, len(self.problem.f)
        rows_to_subtract = []
        # variables in a row always go in the following order:
        # x_1, x_2, ..., x_n, slack variables, excess variables, artificial variables, b
        for i, constraint in enumerate(self.problem.constraints):
            row = constraint.coefs
            if constraint.type == "leq":
                row += [0] * cur_leqs + [1] + [0] * (leqs - cur_leqs - 1) + [0] * geqs + [0] * (geqs + eqs) + [constraint.b]
                self.simplex_table.add_row(initial_count + cur_leqs, row)
                cur_leqs += 1
            elif constraint.type == "geq":
                row += [0] * leqs + [0] * cur_geqs + [-1] + [0] * (geqs - cur_geqs - 1) + [0] * (cur_geqs + cur_eqs) + [1] + [0] * (geqs + eqs - cur_geqs - cur_eqs - 1) + [constraint.b]
                self.simplex_table.add_row(initial_count + leqs + geqs + cur_geqs + cur_eqs, row)
                rows_to_subtract.append(i)
                cur_geqs += 1
            else:
                row += [0] * leqs + [0] * geqs + [0] * (cur_geqs + cur_eqs) + [1] + [0] * (geqs + eqs - cur_geqs - cur_eqs - 1) + [constraint.b]
                self.simplex_table.add_row(initial_count + leqs + geqs + cur_geqs + cur_eqs, row)
                rows_to_subtract.append(i)
                cur_eqs += 1
        self.simplex_table.add_row(-1, [0] * (initial_count + leqs + geqs) + [1] * (geqs + eqs) + [0])

        for row in rows_to_subtract:
            self.simplex_table.subtract_rows(-1, row)

        if self.disp:
            print("Initial table for phase 1:")
            self.simplex_table.print()


In [74]:
with open("task2.json") as f:
    data = json.loads(f.read())
    constraints = list(map(lambda c: Constraint(**c), data["constraints"]))
    problem = LinearProgram(data["f"], data["goal"], constraints)
print(problem)
solver = SimplexSolver(problem)
solver.solve(disp=True)

Problem:
6x_1 + x_2 -> max

Constraints:
-x_1 + 3x_2 ≤ 6
x_1 - 3x_2 = 6
x_1 + x_2 ≥ 1

Initial table for phase 1:
+-------+-----+-----+-----+-----+-----+-----+----+
| Basis | x_1 | x_2 | x_3 | x_4 | x_5 | x_6 | b  |
+-------+-----+-----+-----+-----+-----+-----+----+
|  x_3  |  -1 |  3  |  1  |  0  |  0  |  0  | 6  |
|  x_5  |  1  |  -3 |  0  |  0  |  1  |  0  | 6  |
|  x_6  |  1  |  1  |  0  |  -1 |  0  |  1  | 1  |
|   f   |  -2 |  2  |  0  |  1  |  0  |  0  | -7 |
+-------+-----+-----+-----+-----+-----+-----+----+
